<a href="https://colab.research.google.com/github/hunterkasparian/DiffusionGraph/blob/main/Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.io as pio
from scipy.linalg import solve_banded
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from typing import no_type_check
import random
import plotly.express as px
import pandas as pd
pio.templates.default = 'plotly_dark'

In [38]:
# the code for my way of diffusion
# evaluating diffusion for length nx with sum of particlees @ position
nx = 100 # length
nt = 5000 # time steps
T = np.full(nx,0)
T[0] = 1000 # end concentrations are fixed
x = np.arange(nx) # x axis

fig = make_subplots(rows=1,cols=1)
fig.add_scatter(x=x, y = T, mode= 'lines', name='Initial')

for i in range(1,nt):
    pmovementL = np.zeros(nx) 
    pmovementR = np.zeros(nx)

    for n in range(1,len(T)-1): # calculating probability of movement ~random~ evaluated per position
      if T[n-1] != 0:
        # each particle at a position can either move left (value=1) or move right (value=0)
        # probability of movement evaluated at each position = sum(random # particles that move) / total particles
        pmovementL[n-1] = np.sum ( np.random.choice ([0,1] , (T[n-1]) ) ) / T[n-1] 
        # probability of moving right = 1 - probability moving left
        pmovementR[n-1] = 1.0 - pmovementL[n-1]

    # never need to update the ends, values are always max concentration & 0
    T[1:-1] = pmovementR[2:] * T[2:] + pmovementL[:-2] * T[:-2] # particles on right * prob move right + particles on left * prob move left

    if i % 100 == 0:
      fig.add_scatter(x=x, y = T, mode= 'lines', name=f'{i}')
fig.show()

In [39]:
# evaluating each particle in one long array
# normal distribution
concentration = 10000
active = np.full(concentration, 0)
time = 200
for x in range(time):
  movement = np.random.choice ( [-1,1] , len(active) ) # creates an array length the number of particles and decides a movement for each particle
  active = active + movement 

fig = px.histogram(active, nbins=100)
fig.show()

In [40]:
# evaluating each particle in one long array
active = np.full(1000,0)
time = 20000
flux = []
x = np.arange(time) # x axis for flux scatter plot
for i in range(time):
  movement = np.random.choice ( [-1,1] , len(active) ) # creates an array length the number of particles and decides a movement for each particle
  active = active + movement 
  active = active[active != 0 ] # removes all the starting values to replace them
  active = active[active != -1 ] # removes all the values at position -1 (boundary condition)
  active = np.append(active, np.full(1000,0)) # restates the initial particles conditions
  flux = np.append(flux, len(active[active == 100]))
  active = active[active != 100] # removes the particles that exited the codition

# Displaying concentration x-axis = position , y-axis = total # particles
fig = px.histogram(active, nbins=100)
fig.show()

In [41]:
# Displaying
fig1 = make_subplots(rows=1,cols=1)
fig1.add_scatter(x=x, y = flux, mode= 'lines', name = 'flux')
fig1.show()